# Explore elements of galaxy survey design using the heirarchical model

In this example, we'll be exploring some elements of what is important and effective when designing surveys, using full arbitrary mass measurement uncertainties.

In [1]:
# Imports
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from mrpy.fit_perobj import fit_perobj_stan
from mrpy import TGGD

Our main question will be this:

> Supposing we keep the number of galaxies surveyed fixed at *N*, is it better for cosntraints on the MRP if these
> galaxies are deep (i.e. low truncation mass with small volume) or shallow (i.e. high truncation mass with large volume)?

The answer to the question is *a prior* not obvious. While making the truncation mass lower will increase the range we can access information in our fit, the higher-mass objects are inherently less uncertain in their measurement, and could control the all-important pivot region.

Our initial attack at the problem will be to generate a series of samples with differing truncation masses, but with the same number of galaxies, and fit each sample using our heirarchical Stan model. Importantly, we must use a relation for the uncertainty in the masses, as a function of mass. This will be gotten from data....

## Data Creation

Let's begin by creating intrinsic samples for each selection:

In [2]:
# The basic parameters of our runs
N = 50000
mmins = np.linspace(10.0,13.5,7)

# MRP parameters
logHs = 14.5
alpha = -1.9
beta  = 0.75

In [3]:
intrinsic_masses = np.zeros((N,len(mmins)))

np.random.seed(1234)

for i,mmin in enumerate(mmins):
    intrinsic_masses[:,i] = np.log10(TGGD(10**logHs,alpha,beta,10**mmin).rvs(N))

Now add uncertainty to each sample, based on our uncertainty model

In [ ]:
# adding uncertainty...
#measured_masses = measure(intrinsic_masses,seed=1111)

## Setting Up the Stan Model

First specify some general parameters to use in the estimation:

In [4]:
warmup = 500
iters = 1500
chains = 4

Now define a function that will fit a given sample:

In [5]:
def run_fit(masses,mmin):
    init = [{"logHs":logHs,"alpha":alpha,"beta":beta,"log_mtrue":masses,"log_mmin":masses.min()-0.01}]*chains
    
    # Run the sampling
    fit_err = fit_perobj_stan(10**masses,sd,warmup=warmup,iter=iters,hs_bounds=(logHs-2,logHs+2),
                              alpha_bounds=(-1.99,-1.5),beta_bounds=(beta-0.3,beta+0.3),
                              mmin_bounds=(11,13),n_jobs=chains,chains=chains,init=init,
                              pars=['logHs',"alpha","beta",'log_mmin'],model=model)

Now fit each sample:

In [ ]:
#for i,mmin in enumerate(mmins):
#    result = run_fit()